In [ ]:
!pip install optuna

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import gc
import random
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix,roc_auc_score



In [5]:
from sklearn.impute   import SimpleImputer

train1 = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/test.csv')
columns = [col for col in train1.columns.to_list() if col not in ['id','song_popularity']]
data=train1[columns]
target=train1['song_popularity']

median_impute  = SimpleImputer(strategy='median')
imputed_data = median_impute.fit_transform(data)
imputed_data = pd.DataFrame(imputed_data, columns = data.columns)
train=imputed_data

In [6]:
NUM_CLASSES=target.nunique()

In [7]:
train

,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,212990.0,0.642286,0.856520,0.707073,0.002001,10.0,0.135945,-5.619088,0.0,0.082570,158.386236,4.0,0.734642
1,186660.0,0.054866,0.733289,0.835545,0.000996,8.0,0.436428,-5.236965,1.0,0.127358,102.752988,3.0,0.711531
2,193213.0,0.140532,0.188387,0.783524,-0.002694,5.0,0.170499,-4.951759,0.0,0.052282,178.685791,3.0,0.425536
3,249893.0,0.488660,0.585234,0.552685,0.000608,0.0,0.094805,-7.893694,0.0,0.035618,128.715630,3.0,0.453597
4,165969.0,0.493017,0.608234,0.740982,0.002033,10.0,0.094891,-2.684095,0.0,0.050746,121.928157,4.0,0.741311
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,237799.0,0.140532,0.748942,0.511234,0.002913,0.0,0.173803,-8.853673,0.0,0.078060,168.262924,3.0,0.178159
39996,191119.0,0.067488,0.672830,0.889685,0.001225,5.0,0.122924,-7.798993,0.0,0.188607,110.684544,3.0,0.790626
39997,160879.0,0.877431,0.409065,0.292671,0.001974,10.0,0.110664,-15.378585,0.0,0.031294,99.556074,3.0,0.177947
39998,193918.0,0.140532,0.365738,0.704453,0.000339,1.0,0.356308,-4.661977,1.0,0.054096,139.857384,3.0,0.772978


In [8]:
params={'alpha': 0.4275932000556265,
 'colsample_bytree': 0.3303272480369658,
 'eval_metric': 'mlogloss',
 'lambda': 0.013685373489365672,
 'learning_rate': 0.08882012188858979,
 'max_depth': 17,
 'min_child_weight': 223,
 'num_class': 2,
 'objective': 'multi:softprob',
 'subsample': 0.5807927818586308,
 'tree_method': 'gpu_hist'}

In [9]:
from tqdm import tqdm
np.random.seed(64)
random.seed(64)


target = train1['song_popularity'].values
FOLDS=10
kf = StratifiedKFold(n_splits = FOLDS, shuffle=True, random_state=64)    
        
oof = np.zeros((train.shape[0],))
predictions = 0
auc_value = []
for f, (train_index, test_index) in tqdm(enumerate(kf.split(X=train, y=target))):
        X_train, X_val = train.iloc[train_index][columns], train.iloc[test_index][columns]
        y_train, y_val = target[train_index], target[test_index]
        
        model = xgb.XGBClassifier(**params)
        
        model.fit(
            X_train[columns], 
            y_train,
            eval_set=[(X_val[columns], y_val)],
            early_stopping_rounds=30,
            verbose=100
        )
        
        oof_temp_pred = model.predict_proba(X_val[columns])[:,1]
        test_temp_pred = model.predict_proba(test[columns])[:,1]
        oof[test_index] = oof_temp_pred
        predictions += test_temp_pred/FOLDS
        auc = roc_auc_score(y_val, oof_temp_pred)
        auc_value.append(auc)
        print(f'FOLD: {f} AUC: {auc} Mean AUC: {np.mean(auc_value)}')

0it [00:00, ?it/s]

[0]	validation_0-mlogloss:0.686511
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
[99]	validation_0-mlogloss:0.647932


1it [00:01,  1.70s/it]

FOLD: 0 AUC: 0.5761713895061227 Mean AUC: 0.5761713895061227
[0]	validation_0-mlogloss:0.686195
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[59]	validation_0-mlogloss:0.648694



2it [00:02,  1.33s/it]

...

FOLD: 7 AUC: 0.572743883551938 Mean AUC: 0.5758578012931213
[0]	validation_0-mlogloss:0.686462
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
Stopping. Best iteration:
[62]	validation_0-mlogloss:0.649602



9it [00:09,  1.04s/it]

FOLD: 8 AUC: 0.5656703998342254 Mean AUC: 0.5747258677976883
[0]	validation_0-mlogloss:0.686187
Will train until validation_0-mlogloss hasn't improved in 30 rounds.
[99]	validation_0-mlogloss:0.644495


10it [00:10,  1.09s/it]

FOLD: 9 AUC: 0.5909823335588991 Mean AUC: 0.5763515143738094


In [10]:
submission_df =pd.read_csv('/content/gdrive/MyDrive/song-popularity-prediction/sample_submission.csv')
submission_df['song_popularity'] = predictions
submission_df.to_csv('submission_xgb_best_params1_250_trials_imputed_median.csv', index=False)